In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

from common.models import simple_conv
import numpy as np



c:\Users\Zumo\anaconda3\envs\torch-gpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Zumo\anaconda3\envs\torch-gpu\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\Zumo\anaconda3\envs\torch-gpu\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
bs = 4
tl = 1024
in_feats = 460
base_width = 128
down_layers = 3
dev = torch.device("cuda")

encoder = simple_conv.Encoder(in_feats, down_layers, base_width, [False, True, True])

# model = simple_conv.SimpleSegConv(encoder, 64)
model = simple_conv.SimpleConv(encoder, 64, 1024).to(dev)


In [4]:
ins = torch.randn(bs, tl, in_feats, device=dev)
ins.size()

torch.Size([4, 1024, 460])

In [43]:
_res = []
for _ in range(1000):
    _res.extend(model(ins).cpu().detach().numpy())
res = np.concatenate(_res)
res_t = torch.as_tensor(res)

In [44]:
res.shape, res_t.size()

((4096000, 64), torch.Size([4096000, 64]))

In [14]:
from sklearn.neighbors import KernelDensity

n, d = res.shape
h = (n * (d + 2) / 4.)**(-1. / (d + 4))

kde = KernelDensity(kernel="gaussian", bandwidth=h)

In [46]:
kde.fit(res)

KernelDensity(bandwidth=0.7935259235952904)

In [45]:
from common.kde import KernelDensityEstimator, GaussianKernel

kde2 = KernelDensityEstimator(res_t, GaussianKernel(h), dev)

In [52]:
kde.score_samples(res[:100])

In [48]:
kde2.score_samples(res_t[:10])

RuntimeError: CUDA out of memory. Tried to allocate 9.77 GiB (GPU 0; 6.00 GiB total capacity; 3.54 GiB already allocated; 736.52 MiB free; 3.56 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [51]:
kde2.kernel(res_t[:100], res_t)

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 104857600000 bytes.

In [49]:
def test(test_data: torch.Tensor, train_data: torch.Tensor) -> None:
    """Computes difference between each x in test_data with all train_data."""

    z = kde2.kernel._get_z(train_data)
    print(z)
    train_data = train_data.unsqueeze(0)

    rets = []
    for datum in test_data:
        diffs = datum - train_data
        log_exp = -0.5 * torch.norm(diffs, p=2, dim=-1) ** 2 

        fin = torch.logsumexp(log_exp - z, dim=-1)
        rets.append(fin.item())
    
    rets_t = torch.as_tensor(rets)
    print(rets_t.size())

In [50]:
test(res_t[:10].to(dev), res_t.to(dev))

tensor(59.2364, device='cuda:0')


RuntimeError: CUDA out of memory. Tried to allocate 1000.00 MiB (GPU 0; 6.00 GiB total capacity; 4.52 GiB already allocated; 0 bytes free; 4.53 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF